In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

In [ ]:
# If you're not using Colab, you might need to upgrade jupyter notebook to avoid the following error:
# 'ImportError: IProgress not found. Please update jupyter and ipywidgets.'

! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

# Please restart the kernel after running this cell

In [ ]:
import json
import os

from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager
from nemo.utils import logging
from omegaconf import OmegaConf
import pandas as pd
import pytorch_lightning as pl
import torch
import wget 

# Task description

Intent recognition is the task of classifying the intent of an utterance or document. For example,  for the query:  `What is the weather in Santa Clara tomorrow morning?`, we would like to classify the intent as `weather`. This is a fundamental step that is executed in any task-driven conversational assistant.

Typical text classification models, such as the Joint Intent and Slot Classification Model in NeMo, are trained on hundreds or thousands of labeled documents. In this tutorial we demonstrate a different, "zero shot" approach that requires no annotated data for the target intents. The zero shot approach uses a model trained on the task of natural language inference (NLI). During training, the model is presented with pairs of sentences consisting of a "premise" and a "hypothesis", and must classify the relationship between them as entailment (meaning the hypothesis follows logically from the premise), contradiction, or neutral. To use this model for intent prediction, we define a list of candidate labels to represent each of the possible classes in our classification system; for example, the candidate labels might be `request for directions`,  `query about weather`, `request to play music`, etc. We predict the intent of a query by pairing it with each of the candidate labels as a premise-hypothesis pair and using the model to predict the probability of an entailment relationship between them. For example, for the query and candidate labels above, we would run inference for the following pairs:

(`What is the weather in Santa Clara tomorrow morning?`, `request for directions`)  
(`What is the weather in Santa Clara tomorrow morning?`, `request to play music`)   
(`What is the weather in Santa Clara tomorrow morning?`, `query about weather`)

In the above example, we would expect a high probability of entailment for the last pair, and low probabilities for the first two pairs. Thus, we would classify the intent of the utterance as `query about weather`. The task can be formulated as single-label classification (only one of the candidate labels can be correct for each query) or multi-label classification (multiple labels can be correct) by setting the parameter multi_label = False or multi_label = True, respectively, during inference.

In this tutorial, we demonstrate how to train an NLI model on the MNLI data set and how to use it for zero shot intent recognition.

# Using an out-of-the-box model

In [ ]:
# this line will download a pre-trained NLI model from NVIDIA's NGC cloud and instantiate it for you

pretrained_model = nemo_nlp.models.ZeroShotIntentModel.from_pretrained("zeroshotintent_en_bert_base_uncased")

In [ ]:
queries = [
    "What is the weather in Santa Clara tomorrow morning?",
    "I'd like a veggie burger and fries",
    "Bring me some ice cream when it's sunny"

]

candidate_labels = ['Food order', 'Weather query', "Play music"]

predictions = pretrained_model.predict(queries, candidate_labels, batch_size=4, multi_label=True)

print('The prediction results of some sample queries with the trained model:')
for query in predictions:
    print(json.dumps(query, indent=4))

In the example above, we set `multi_label=True`, which is also the default setting. This runs a softmax calculation independently for each label over the entailment and contradiction logits. For any given query, the scores for the different labels may add up to more than one.

Below, we see what happens if we set `multi_label=False`. In this case, the softmax calculation for each query uses the entailment class logits for all the labels, so the final scores for all classes add up to one.

In [ ]:
predictions = pretrained_model.predict(queries, candidate_labels, batch_size=4, multi_label=False)

print('The prediction results of some sample queries with the trained model:')
for query in predictions:
    print(json.dumps(query, indent=4))

Under the hood, during inference the candidate labels are not used as is; they're actually used to fill in the blank in a hypothesis template. By default, the hypothesis template is `This example is {}`. So the candidate labels above would actually be presented to the model as `This example is food order`, `This example is weather query`, and `This example is play music`. You can change the hypothesis template with the optional keyword argument `hypothesis_template`, as shown below.

In [ ]:
predictions = pretrained_model.predict(queries, candidate_labels, batch_size=4, multi_label=False,
                                      hypothesis_template="a person is asking something related to {}")

print('The prediction results of some sample queries with the trained model:')
for query in predictions:
    print(json.dumps(query, indent=4))

Now, let's take a closer look at the model's configuration and learn to train the model.


# Training your own model


# Dataset

In this tutorial we will train a model on [The Multi-Genre Natural Language Inference Corpus](https://cims.nyu.edu/~sbowman/multinli/multinli_0.9.pdf) (MNLI). This is a crowdsourced collection of sentence pairs with textual entailment annotations. Given a premise sentence followed by a hypothesis sentence, the task is to predict whether the premise entails the hypothesis (entailment), contradicts the hypothesis (contradiction), or neither (neutral). There are two dev sets for this task: the "matched" dev set contains examples drawn from the same genres as the training set, and the "mismatched" dev set has examples from genres not seen during training. For our purposes, either dev set alone will be sufficient. We will use the "matched" dev set here. 

## Download the dataset

In [ ]:
# you can replace DATA_DIR with your own location
DATA_DIR = '.'  

In [ ]:
wget.download('https://dl.fbaipublicfiles.com/glue/data/MNLI.zip', DATA_DIR)
! unzip {DATA_DIR}/MNLI.zip -d {DATA_DIR}

In [ ]:
! ls -l $DATA_DIR/MNLI

We will use `train.tsv` as our training set and `dev_matched.tsv` as our validation set.

## Explore the dataset  
Let's take a look at some examples from the dev set

In [ ]:
num_examples = 5
df = pd.read_csv(os.path.join(DATA_DIR, "MNLI", "dev_matched.tsv"), sep="\t")[:num_examples]
for sent1, sent2, label in zip(df['sentence1'].tolist(), df['sentence2'].tolist(), df['gold_label'].tolist()):
    print("sentence 1: ", sent1)
    print("sentence 2: ", sent2)
    print("label: ", label)
    print("===================")

# Training model
## Model configuration

The model is comprised of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model followed by a Sequence Classifier module.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, a classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

All model and training parameters are defined in the **zero_shot_intent_config.yaml** config file. This file is located in the folder **examples/nlp/zero_shot_intent_recognition/conf/**. It contains 2 main sections:


We will download the config file from the repository for the purpose of the tutorial. If you have a version of NeMo installed locally, you can use it from the above folder.

In [ ]:
# download the model config file from repository for the purpose of this example
WORK_DIR = "."  # you can replace WORK_DIR with your own location
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/zero_shot_intent_recognition/conf/zero_shot_intent_config.yaml', WORK_DIR)

# print content of the config file
config_file = os.path.join(WORK_DIR, "zeroshot_intent_config.yaml")
config = OmegaConf.load(config_file)
print(OmegaConf.to_yaml(config))

## Setting up data within the config

Among other things, the config file contains dictionaries called **dataset**, **train_ds** and **validation_ds**. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

To start model training, we need to specify `model.dataset.data_dir`, `model.train_ds.file_name` and `model.validation_ds.file_name`, as we are going to do below.

Notice that some config lines, including `model.train_ds.data_dir`, have `???` in place of paths. This means that values for these fields are required to be specified by the user.

Let's now add the data paths and output directory for saving predictions to the config.

In [ ]:
# you can replace OUTPUT_DIR with your own location; this is where logs and model checkpoints will be saved
OUTPUT_DIR = "nemo_output"
config.exp_manager.exp_dir = OUTPUT_DIR
config.model.dataset.data_dir = os.path.join(DATA_DIR, "MNLI")
config.model.train_ds.file_name = "train.tsv"
config.model.validation_ds.file_path = "dev_matched.tsv"

## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

config.trainer.precision = 16 if torch.cuda.is_available() else 32

# for mixed precision training, uncomment the line below (precision should be set to 16 and amp_level to O1):
# config.trainer.amp_level = O1

# remove distributed training flags
config.trainer.accelerator = None

# setup max number of steps to reduce training time for demonstration purposes of this tutorial
config.trainer.max_steps = 128

trainer = pl.Trainer(**config.trainer)

## Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [ ]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model and use [Megatron-LM BERT](https://arxiv.org/abs/1909.08053) or [AlBERT model](https://arxiv.org/abs/1909.11942):

In [ ]:
# get the list of supported BERT-like models, for the complete list of HugginFace models, see https://huggingface.co/models
print(nemo_nlp.modules.get_pretrained_lm_models_list(include_external=True))

# specify BERT-like model, you want to use, for example, "megatron-bert-345m-uncased" or 'bert-base-uncased'
PRETRAINED_BERT_MODEL = "albert-base-v1"

In [ ]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [ ]:
model = nemo_nlp.models.ZeroShotIntentModel(cfg=config.model, trainer=trainer)

## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.

In [ ]:
try:
  from google import colab
  COLAB_ENV = True
except (ImportError, ModuleNotFoundError):
  COLAB_ENV = False

# Load the TensorBoard notebook extension
if COLAB_ENV:
  %load_ext tensorboard
  %tensorboard --logdir {exp_dir}
else:
  print("To use tensorboard, please use this notebook in a Google Colab environment.")

In [ ]:
# start model training
trainer.fit(model)

## Inference from Examples
The next step is to see how the trained model will classify intents. To improve the predictions you may need to train the model for more than 5 epochs.



In [ ]:
# reload the saved model
saved_model = os.path.join(exp_dir, "checkpoints/ZeroShotIntentRecognition.nemo")
eval_model = nemo_nlp.models.ZeroShotIntentModel.restore_from(saved_model)

In [ ]:
queries = [
    "I'd like a veggie burger and fries",
    "Turn off the lights in the living room",
]

candidate_labels = ['Food order', 'Play music', 'Request for directions', 'Change lighting', 'Calendar query']

predictions = eval_model.predict(queries, candidate_labels, batch_size=4, multi_label=True)

print('The prediction results of some sample queries with the trained model:')
for query in predictions:
    print(json.dumps(query, indent=4))
print("Inference finished!")

As described above in "Using an out of the box model", you can set multi_label=False if you want the scores for each query to add up to one. You can also change the hypothesis template used when presenting candidate labels, as shown below.

In [ ]:
predictions = eval_model.predict(queries, candidate_labels, batch_size=4, multi_label=True,
                           hypothesis_template="related to {}")

print('The prediction results of some sample queries with the trained model:')
for query in predictions:
    print(json.dumps(query, indent=4))
print("Inference finished!")

By default, when an NLI model is trained on MNLI in NeMo, the class indices for entailment and contradiction are 1 and 0, respectively. The `predict` method uses these indices by default. If your NLI model was trained with different class indices for these classes, you can pass the correct indices as keyword arguments to the `predict` method (e.g. `entailment_idx=1`, `contradiction_idx=0`). 

## Training Script

If you have NeMo installed locally, you can also train the model with [examples/nlp/zero_shot_intent_recognition/zero_shot_intent_train.py](https://github.com/carolmanderson/NeMo/blob/main/examples/nlp/zero_shot_intent_recognition/zero_shot_intent_train.py).

To run training script, use:

```
python zero_shot_intent_train.py \
 model.dataset.data_dir=PATH_TO_DATA_FOLDER
```
 
 By default, this script uses `examples/nlp/zero_shot_intent_recognition/conf/zero_shot_intent_config.yaml` config file, and you may update all the params inside of this config file or alternatively provide them in the command line.
